In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

In [ ]:
POSSIBLE_VALUES={1,2,3,4,5,6,7,8,9}

In [29]:
#easy puzzle
input_board=pd.DataFrame(
    np.array(
        [
            [9,np.NaN,np.NaN,5,2,np.NaN,3,7,np.NaN],
            [4,np.NaN,np.NaN,7,np.NaN,6,np.NaN,np.NaN,2],
            [7,np.NaN,6,np.NaN,np.NaN,np.NaN,5,np.NaN,np.NaN],
            [np.NaN,1,9,3,5,np.NaN,np.NaN,np.NaN,np.NaN,],
            [np.NaN,3,np.NaN,4,np.NaN,1,np.NaN,2,np.NaN],
            [np.NaN,np.NaN,np.NaN,np.NaN,8,7,1,6,np.NaN],
            [np.NaN,np.NaN,4,np.NaN,np.NaN,np.NaN,6,np.NaN,8],
            [2,np.NaN,np.NaN,1,np.NaN,3,np.NaN,np.NaN,9],
            [np.NaN,7,5,np.NaN,9,8,np.NaN,np.NaN,1],
        ]
    ),
)
input_board=input_board.astype(object)

In [2]:
#hard puzzle - broken puzzle?
input_board=pd.DataFrame(
    np.array(
        [
            [np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,9],
            [np.NaN,8,np.NaN,5,6,np.NaN,np.NaN,np.NaN,np.NaN],
            [np.NaN,np.NaN,np.NaN,np.NaN,2,np.NaN,1,np.NaN,4],
            [np.NaN,6,np.NaN,np.NaN,np.NaN,8,np.NaN,np.NaN,np.NaN],
            [4,5,np.NaN,np.NaN,9,np.NaN,np.NaN,1,2],
            [np.NaN,np.NaN,np.NaN,3,np.NaN,np.NaN,np.NaN,7,np.NaN],
            [1,np.NaN,9,np.NaN,4,np.NaN,np.NaN,np.NaN,np.NaN],
            [np.NaN,np.NaN,np.NaN,np.NaN,7,6,np.NaN,3,np.NaN],
            [7,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN]
        ]
    ),
)
input_board=input_board.astype(object)

In [3]:
input_board

,0,1,2,3,4,5,6,7,8
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0
1,NaN,8.0,NaN,5.0,6.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN,4.0
3,NaN,6.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN
4,4.0,5.0,NaN,NaN,9.0,NaN,NaN,1.0,2.0
5,NaN,NaN,NaN,3.0,NaN,NaN,NaN,7.0,NaN
6,1.0,NaN,9.0,NaN,4.0,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,7.0,6.0,NaN,3.0,NaN
8,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def determine_row_preclusions(sudoku_board, row_coordinate):
    row_preclusions=set(sudoku_board.loc[row_coordinate,:].dropna().unique())
    return row_preclusions

In [6]:
def determine_column_preclusions(sudoku_board, column_coordinate):
    column_preclusions=set(sudoku_board.loc[:,column_coordinate].dropna().unique())
    return column_preclusions

In [7]:
def determine_coordinate_slice_edges(coordinate):
    if coordinate<3:
        coordinate_min=0
        coordinate_max=2
    elif coordinate>=3 and coordinate <6:
        coordinate_min=3
        coordinate_max=5
    elif coordinate >=6:
        coordinate_min=6
        coordinate_max=8
        
    return coordinate_min, coordinate_max

In [8]:
def determine_mini_square_preclusions(sudoku_board, row_coordinate, column_coordinate):
    row_min,row_max=determine_coordinate_slice_edges(row_coordinate)
    col_min,col_max=determine_coordinate_slice_edges(column_coordinate)
    
    sudoku_board_mini_square=sudoku_board.loc[
        row_min:row_max,
        col_min:col_max
    ]
    
    mini_square_preclusions=set(sudoku_board_mini_square.stack().dropna().unique())
    
    return mini_square_preclusions

In [9]:
def determine_possibilities(sudoku_board):
    
    new_board_list=list()
    
    #treating i as rows
    for i in range(len(POSSIBLE_VALUES)):
        
        new_row_list=list()
        
        for j in range(len(POSSIBLE_VALUES)):
            
            if pd.isnull(sudoku_board.at[i,j])==True:
                row_preclusions=determine_row_preclusions(sudoku_board,i)
                column_preclusions=determine_column_preclusions(sudoku_board,j)
                mini_square_preclusions=determine_mini_square_preclusions(sudoku_board,i,j)
                total_preclusions=row_preclusions.union(column_preclusions).union(mini_square_preclusions)
                possibilities=POSSIBLE_VALUES.difference(total_preclusions)
            else:
                possibilities={int((sudoku_board.at[i,j]))}

            new_row_list.append(possibilities)    
            
        new_board_list.append(new_row_list)
        
    return pd.DataFrame(new_board_list)
            

In [10]:
def determine_initial_determined_coordinates(sudoku_board):
    return sudoku_board.applymap(
        lambda x: 1 if pd.isnull(x)==False else 0
    )

In [11]:
def determine_determined_coordinates(sudoku_board):
    return sudoku_board.applymap(
        lambda x: 1 if len(x)==1 else 0
    )

In [15]:
def update_row_possibilities(board_possibilities,row_coordinate,column_coordinate,impossible_value):
    for j in range(len(POSSIBLE_VALUES)):
        board_possibilities.at[
            row_coordinate,
            j
        ]=board_possibilities.at[row_coordinate,j].difference({impossible_value})
        
    board_possibilities.at[
        row_coordinate,
        column_coordinate
    ]=board_possibilities.at[row_coordinate,column_coordinate].union({impossible_value})
        
    return board_possibilities

In [16]:
def update_column_possibilities(board_possibilities,row_coordinate,column_coordinate,impossible_value):
    for i in range(len(POSSIBLE_VALUES)):
        board_possibilities.at[
            i,
            column_coordinate
        ]=board_possibilities.at[i,column_coordinate].difference({impossible_value})
        
    board_possibilities.at[
        row_coordinate,
        column_coordinate
    ]=board_possibilities.at[row_coordinate,column_coordinate].union({impossible_value})
        
    return board_possibilities

In [31]:
def update_mini_square_possibilities(board_possibilities,row_coordinate,column_coordinate,impossible_value):
    row_min,row_max=determine_coordinate_slice_edges(row_coordinate)
    col_min,col_max=determine_coordinate_slice_edges(column_coordinate)
    
    for i in range(row_min,row_max+1):
        for j in range(col_min,col_max+1):
            board_possibilities.at[
                i,
                j
            ]=board_possibilities.at[i,j].difference({impossible_value})

    board_possibilities.at[
        row_coordinate,
        column_coordinate
    ]=board_possibilities.at[row_coordinate,column_coordinate].union({impossible_value})
        
    return board_possibilities

In [33]:
#governing logic

previous_determined_board=determine_initial_determined_coordinates(input_board)
current_possibilities=determine_possibilities(input_board)

while 1==1:
    display(current_possibilities)
    
    this_iterations_determined_status=determine_determined_coordinates(current_possibilities)
    
    fresh_deduction_locations=this_iterations_determined_status.values-previous_determined_board.values
    
    deduction_coordinates=np.argwhere(fresh_deduction_locations==1)
    
    if len(deduction_coordinates)==0:
        break
    
    for temp_pair in deduction_coordinates:
        deduced_value=current_possibilities.at[temp_pair[0],temp_pair[1]].pop()
        current_possibilities=update_row_possibilities(current_possibilities,temp_pair[0],temp_pair[1],deduced_value)
        current_possibilities=update_column_possibilities(current_possibilities,temp_pair[0],temp_pair[1],deduced_value)
        current_possibilities=update_mini_square_possibilities(current_possibilities,temp_pair[0],temp_pair[1],deduced_value)
        
    previous_determined_board=this_iterations_determined_status


,0,1,2,3,4,5,6,7,8
0,{9},{8},"{8, 1}",{5},{2},{4},{3},{7},"{4, 6}"
1,{4},"{8, 5}","{8, 1, 3}",{7},"{1, 3}",{6},"{8, 9}","{8, 1, 9}",{2}
2,{7},"{8, 2}",{6},"{8, 9}","{1, 3, 4}","{9, 4}",{5},"{8, 1, 4, 9}",{4}
3,"{8, 6}",{1},{9},{3},{5},{2},"{8, 4, 7}","{8, 4}","{4, 7}"
4,"{8, 5, 6}",{3},"{8, 7}",{4},{6},{1},"{8, 9, 7}",{2},"{5, 7}"
5,{5},"{2, 4, 5}",{2},"{9, 2}",{8},{7},{1},{6},"{3, 4, 5}"
6,"{1, 3}",{9},{4},{2},{7},"{2, 5}",{6},"{3, 5}",{8}
7,{2},"{8, 6}",{8},{1},"{4, 6, 7}",{3},"{4, 7}","{4, 5}",{9}
8,"{3, 6}",{7},{5},"{2, 6}",{9},{8},"{2, 4}","{3, 4}",{1}


,0,1,2,3,4,5,6,7,8
0,{9},{8},{1},{5},{2},{4},{3},{7},{6}
1,{4},{5},"{1, 3}",{7},"{1, 3}",{6},"{8, 9}","{8, 1, 9}",{2}
2,{7},{2},{6},"{8, 9}","{1, 3}",{9},{5},"{8, 1, 9}",{4}
3,"{8, 6}",{1},{9},{3},{5},{2},"{8, 4, 7}","{8, 4}",{7}
4,{8},{3},{7},{4},{6},{1},"{8, 9, 7}",{2},"{5, 7}"
5,{5},{4},{2},{9},{8},{7},{1},{6},{3}
6,"{1, 3}",{9},{4},{2},{7},{5},{6},"{3, 5}",{8}
7,{2},{6},{8},{1},{4},{3},"{4, 7}","{4, 5}",{9}
8,"{3, 6}",{7},{5},{6},{9},{8},"{2, 4}","{3, 4}",{1}


,0,1,2,3,4,5,6,7,8
0,{9},{8},{1},{5},{2},{4},{3},{7},{6}
1,{4},{5},{3},{7},"{1, 3}",{6},"{8, 9}","{8, 1, 9}",{2}
2,{7},{2},{6},{8},"{1, 3}",{9},{5},"{8, 1}",{4}
3,{6},{1},{9},{3},{5},{2},"{8, 4}","{8, 4}",{7}
4,{8},{3},{7},{4},{6},{1},{9},{2},{5}
5,{5},{4},{2},{9},{8},{7},{1},{6},{3}
6,"{1, 3}",{9},{4},{2},{7},{5},{6},{3},{8}
7,{2},{6},{8},{1},{4},{3},{7},{5},{9}
8,{3},{7},{5},{6},{9},{8},"{2, 4}","{3, 4}",{1}


,0,1,2,3,4,5,6,7,8
0,{9},{8},{1},{5},{2},{4},{3},{7},{6}
1,{4},{5},{3},{7},{1},{6},{8},"{8, 1, 9}",{2}
2,{7},{2},{6},{8},"{1, 3}",{9},{5},{1},{4}
3,{6},{1},{9},{3},{5},{2},"{8, 4}","{8, 4}",{7}
4,{8},{3},{7},{4},{6},{1},{9},{2},{5}
5,{5},{4},{2},{9},{8},{7},{1},{6},{3}
6,{1},{9},{4},{2},{7},{5},{6},{3},{8}
7,{2},{6},{8},{1},{4},{3},{7},{5},{9}
8,{3},{7},{5},{6},{9},{8},"{2, 4}",{4},{1}


,0,1,2,3,4,5,6,7,8
0,{9},{8},{1},{5},{2},{4},{3},{7},{6}
1,{4},{5},{3},{7},{1},{6},{8},{9},{2}
2,{7},{2},{6},{8},{3},{9},{5},{1},{4}
3,{6},{1},{9},{3},{5},{2},{4},{8},{7}
4,{8},{3},{7},{4},{6},{1},{9},{2},{5}
5,{5},{4},{2},{9},{8},{7},{1},{6},{3}
6,{1},{9},{4},{2},{7},{5},{6},{3},{8}
7,{2},{6},{8},{1},{4},{3},{7},{5},{9}
8,{3},{7},{5},{6},{9},{8},{2},{4},{1}


,0,1,2,3,4,5,6,7,8
0,{9},{8},{1},{5},{2},{4},{3},{7},{6}
1,{4},{5},{3},{7},{1},{6},{8},{9},{2}
2,{7},{2},{6},{8},{3},{9},{5},{1},{4}
3,{6},{1},{9},{3},{5},{2},{4},{8},{7}
4,{8},{3},{7},{4},{6},{1},{9},{2},{5}
5,{5},{4},{2},{9},{8},{7},{1},{6},{3}
6,{1},{9},{4},{2},{7},{5},{6},{3},{8}
7,{2},{6},{8},{1},{4},{3},{7},{5},{9}
8,{3},{7},{5},{6},{9},{8},{2},{4},{1}
